#### Задание 1
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) и задавались следующим образом:   
x - последовательность цифр   
y[0] = x[0],   
y[i] = x[i] + x[0]   
Если y[i] >= 10, то y[i] = y[i] - 10

**Задача:**
1. научить модель предсказывать y(i) по x(i)
2. пробовать RNN, LSTM, GRU

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

##### 1. Генерация данных

In [25]:
def generate_sequence(length):
    x = np.random.randint(0, 10, length)
    y = np.zeros_like(x)
    y[0] = x[0]
    for i in range(1, length):
        y[i] = x[i] + x[0]
        if y[i] >= 10:
            y[i] -= 10
    return x, y

In [26]:
# Генерация набора данных
data_size = 10000
sequence_length = 10

X = np.zeros((data_size, sequence_length), dtype=int)
Y = np.zeros_like(X)
for i in range(data_size):
    x, y = generate_sequence(sequence_length)
    X[i] = x
    Y[i] = y

In [27]:
# Посмотрим набор данных Х
X.shape

(10000, 10)

In [28]:
# Посмотрим набор данных Y
Y.shape

(10000, 10)

##### 2. Подготовка данных
Разделим данные на обучающую и тестовую выборки и преобразуем их в тензоры PyTorch.

In [29]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Преобразование данных в тензоры Pytorch
X_train = torch.Tensor(X_train).unsqueeze(-1) # Добавляем размерность для признаков
X_test = torch.Tensor(X_test).unsqueeze(-1)
Y_train = torch.Tensor(Y_train) 
Y_test = torch.Tensor(Y_test)

In [34]:
# Посмотрим получившуюся матрицу
# Возьмем первый элемент матрицы
print(X[0])
print(Y[0])

[4 0 3 5 7 2 3 1 4 6]
[4 4 7 9 1 6 7 5 8 0]


Проверим условие:   
массив х состоит из случайных чисел от 0 до 9   
y[0] = x[0] = 4   
y[1] = x[1] + x[0] = 0 + 4 = 4   
y[2] = x[2] + x[0] = 3 + 4 = 7   
y[3] = x[3] + x[0] = 5 + 4 = 9   
y[4] = x[4] + x[0] = 7 + 4 = 11 т.к. 11 > 10, тогда y[4] = 11 - 10 = 1   
и т.д.   

Генератор работает правильно.   

In [35]:
# Создадим DataLoader
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

##### 3. Создание модели
Создадим модели RNN, LSTM и GRU

In [36]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class GRU_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_Model, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

class RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_Model, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [37]:
# Параметры модели
input_size = 1
hidden_size = 128
output_size = 10
num_layers = 1

In [38]:
# Инициализируем модели
rnn_model = RNN_Model(input_size, hidden_size, output_size)
lstm_model = LSTM_Model(input_size, hidden_size, output_size)
gru_model = GRU_Model(input_size, hidden_size, output_size) 

##### 4. Обучение моделей

In [43]:
# Функция потерь и оптимизатор
criterion = nn.MSELoss()
learning_rate = 0.001
num_epoch = 15

In [90]:
# Обучим модель
def train_model(model, train_loader, num_epochs=num_epoch):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epoch):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.3f}")

print("Training RNN Model...")
train_model(rnn_model, train_loader)
print('=' * 30)

print("Training LSTM Model...")
train_model(lstm_model, train_loader)
print('=' * 30) 

print("Training GRU Model...")
train_model(gru_model, train_loader)

Training RNN Model...
Epoch [1/15], loss: 8.449
Epoch [2/15], loss: 7.799
Epoch [3/15], loss: 8.110
Epoch [4/15], loss: 8.263
Epoch [5/15], loss: 8.124
Epoch [6/15], loss: 7.935
Epoch [7/15], loss: 7.663
Epoch [8/15], loss: 8.729
Epoch [9/15], loss: 7.903
Epoch [10/15], loss: 8.083
Epoch [11/15], loss: 7.770
Epoch [12/15], loss: 8.058
Epoch [13/15], loss: 7.764
Epoch [14/15], loss: 8.159
Epoch [15/15], loss: 8.210
Training LSTM Model...
Epoch [1/15], loss: 0.007
Epoch [2/15], loss: 0.011
Epoch [3/15], loss: 0.007
Epoch [4/15], loss: 0.010
Epoch [5/15], loss: 0.008
Epoch [6/15], loss: 0.009
Epoch [7/15], loss: 0.009
Epoch [8/15], loss: 0.008
Epoch [9/15], loss: 0.011
Epoch [10/15], loss: 0.009
Epoch [11/15], loss: 0.007
Epoch [12/15], loss: 0.009
Epoch [13/15], loss: 0.010
Epoch [14/15], loss: 0.008
Epoch [15/15], loss: 0.006
Training GRU Model...
Epoch [1/15], loss: 0.004
Epoch [2/15], loss: 0.004
Epoch [3/15], loss: 0.004
Epoch [4/15], loss: 0.004
Epoch [5/15], loss: 0.006
Epoch [6/15

##### 5. Оценка моделей
Протестируем модели на тестовых данных.

In [93]:
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
    avg_loss = total_loss / len(test_loader)
    return avg_loss

In [94]:
# Создаём словарь с данными
data_loss = {
    'Модель': ['RNN', 'LSTM', 'GRU'],
    'Test Loss': [
        evaluate_model(rnn_model, test_loader),
        evaluate_model(lstm_model, test_loader),
        evaluate_model(gru_model, test_loader)
    ]
}

# Создаём датафрейм
df_loss = pd.DataFrame(data_loss)

# Устанавливаем формат отображения чисел
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Выводим датафрейм
print(df_loss.to_string(index=False))

Модель  Test Loss
   RNN      8.287
  LSTM      0.006
   GRU      0.003


##### 6. Предсказание
Используем обученные модели для предсказания.

In [95]:
def predict_sequence(model, x):
    model.eval()
    with torch.no_grad():
        x = torch.tensor(x).unsqueeze(0).unsqueeze(-1)
        y_pred = model(x)
    return y_pred.squeeze().numpy()

# Пример предсказания
x_test = X_test[0].squeeze().numpy()
y_test = Y_test[0].numpy()

y_pred_rnn = predict_sequence(rnn_model, x_test)
y_pred_lstm = predict_sequence(lstm_model, x_test)
y_pred_gru = predict_sequence(gru_model, x_test)

In [96]:
# Создаём словарь с данными
data = {
    'True y': y_test,
    'RNN Predicted y': y_pred_rnn,
    'LSTM Predicted y': y_pred_lstm,
    'GRU Predicted y': y_pred_gru
}

# Создаём датафрейм
df = pd.DataFrame(data)

# Преобразуем 'True y' в целые числа
df['True y'] = df['True y'].astype(int)

In [87]:
print("Predict...")
print(df.to_string(index=False))

Predict...
 True y  RNN Predicted y  LSTM Predicted y  GRU Predicted y
      0            4.711             0.016           -0.128
      4            4.213             4.087            4.018
      6            4.245             5.806            6.462
      6            4.440             5.794            6.066
      9            4.783             8.701            9.024
      0            4.080             0.077            0.055
      8            4.525             7.959            7.804
      6            4.657             6.012            5.930
      1            4.423             0.962            0.979
      7            4.399             7.068            7.048


In [97]:
print("Loss... ")
print(df_loss.to_string(index=False))

Loss... 
Модель  Test Loss
   RNN      8.287
  LSTM      0.006
   GRU      0.003


Из таблицы  predict видно, что модель LSTM и GRU, показали более лучший результат вероятностей. Если посмотреть на Loss, то GRU выглядит немного лучше.   
На мой взгляд, обе модели LSTM и GRU показали качественный результат.

#### Задание 2 (дополнительное и необязательное)
применить LSTM для решения лекционного практического [задания](https://colab.research.google.com/drive/1_rNrPHl6sYHNp-xo2G6I_SmRGTSSeICL)